### Python

In [2]:
import pandas as pd

In [ ]:
data = pd.read_csv("C:\\Users\\carlo\\Documents\\AI\\apartments.csv")

# C:\\Users\\johnb\\Documents\\GitHub\\causalai-2025\\python\\input\\apartments.csv

In [4]:
data.columns

Index(['id', 'price', 'month', 'area', 'type', 'rooms', 'centredistance',
       'schooldistance', 'clinicdistance', 'postofficedistance',
       'kindergartendistance', 'restaurantdistance', 'collegedistance',
       'pharmacydistance', 'ownership', 'buildingmaterial', 'hasparkingspace',
       'hasbalcony', 'haselevator', 'hassecurity', 'hasstorageroom'],
      dtype='object')

In [ ]:
data.head()

In [ ]:
data['area2'] = data['area'] ** 2

In [ ]:
data['hasparkingspace'] = np.where(data['hasparkingspace'] == "yes", 1, 0)
data['hasbalcony'] = np.where(data['hasbalcony'] == "yes", 1, 0)
data['haselevator'] = np.where(data['haselevator'] == "yes", 1, 0)
data['hassecurity'] = np.where(data['hassecurity'] == "yes", 1, 0)
data['hasstorageroom'] = np.where(data['hasstorageroom'] == "yes", 1, 0)

In [ ]:
data['ud'] = data['area'] % 10

In [ ]:
uds = pd.get_dummies(data['ud'], prefix = 'ud')

In [ ]:
data = pd.concat([data, uds], axis=1)

In [ ]:
data['month'] = data['month'].astype('category')
data['type'] = data['type'].astype('category')
data['rooms'] = data['rooms'].astype('category')
data['ownership'] = data['ownership'].astype('category')
data['buildingmaterial'] = data['buildingmaterial'].astype('category')

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
reg = ols(
    """price ~ ud_0 + ud_1 + ud_2 + ud_3 + ud_4 + ud_5 + ud_6 + ud_7 + ud_8
              + area + area2
              + schooldistance + clinicdistance + postofficedistance
              + kindergartendistance + collegedistance + pharmacydistance
              + C(ownership) + C(buildingmaterial)
              + hasparkingspace + hasbalcony + haselevator
              + hassecurity + hasstorageroom""",
    data=data
).fit()

print(reg.summary())

### Partialing Out

In [ ]:
residuos_mc = reg.resid

In [ ]:
modelsinud0 = ols(
    """price ~ ud_1 + ud_2 + ud_3 + ud_4 + ud_5 + ud_6 + ud_7 + ud_8
              + area + area2
              + schooldistance + clinicdistance + postofficedistance
              + kindergartendistance + collegedistance + pharmacydistance
              + C(ownership) + C(buildingmaterial)
              + hasparkingspace + hasbalcony + haselevator
              + hassecurity + hasstorageroom""",
    data=data
).fit()

print(modelsinud0.summary())

In [ ]:
residuos_msinud0 = modelsinud0.resid

In [ ]:
residuos_msinud0.shape

In [ ]:
data["ud_0"] = data["ud_0"].astype(int)

In [ ]:
modelud0 = ols(
    """ud_0 ~ ud_1 + ud_2 + ud_3 + ud_4 + ud_5 + ud_6 + ud_7 + ud_8
              + area + area2
              + schooldistance + clinicdistance + postofficedistance
              + kindergartendistance + collegedistance + pharmacydistance
              + C(ownership) + C(buildingmaterial)
              + hasparkingspace + hasbalcony + haselevator
              + hassecurity + hasstorageroom""",
    data=data
).fit()

print(modelud0.summary())

In [ ]:
residuosud0 = modelud0.resid

In [ ]:
residuosud0.shape

In [ ]:
df_res = pd.DataFrame({
    "residuos_msinud0": modelsinud0.resid,
    "residuosud0": modelud0.resid
})

print(df_res.shape)

p0model = smf.ols("residuos_msinud0 ~ residuosud0", data=df_res).fit()
print(p0model.summary())


In [ ]:
reg.params[1]

In [ ]:
p0model.params[1]

In [ ]:
datasinud0 = data[data["ud_0"] == 0]

In [ ]:
modelnoud0 = smf.ols(
    """price ~ ud_1 + ud_2 + ud_3 + ud_4 + ud_5 + ud_6 + ud_7 + ud_8
              + area + area2
              + schooldistance + clinicdistance + postofficedistance
              + kindergartendistance + collegedistance + pharmacydistance
              + C(ownership) + C(buildingmaterial)
              + hasparkingspace + hasbalcony + haselevator
              + hassecurity + hasstorageroom""",
    data=datasinud0
).fit()

print(modelnoud0.summary())

In [ ]:
data["price_predicted"] = modelnoud0.predict(data)

In [ ]:
price_predicted = modelnoud0.predict(data)

data["price_predicted"] = price_predicted

df_pred_ud0 = pd.DataFrame({
    "price_predicted": price_predicted,
    "ud_0": data["ud_0"]
})

print(df_pred_ud0.head())

In [ ]:
data["price_predicted"] = price_predicted

In [ ]:
dataud0 = data[data["ud_0"] == 1]

In [ ]:
avgprice = dataud0["price"].mean()
avgpredicted = dataud0["price_predicted"].mean(skipna=True)

In [ ]:
Comparison = pd.DataFrame({
    "avgprice": [avgprice],
    "avgpredicted": [avgpredicted]
})

Comparison